In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/credit-card-customer-churn-prediction/Churn_Modelling.csv


In [2]:
df = pd.read_csv('/kaggle/input/credit-card-customer-churn-prediction/Churn_Modelling.csv')

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


# Null values and Duplicate

In [5]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [6]:
df.duplicated().sum()

np.int64(0)

In [7]:
df = df.drop(columns=['RowNumber','CustomerId','Surname'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
df = pd.get_dummies(df, columns=['Geography', 'Gender'], drop_first=True)  #OneHot encoding using the pandas you can perform using the SKlearn

In [9]:
X = df.drop(columns=['Exited'])
y = df['Exited']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
X_train.shape
X_train.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
9254,686,32,6,0.00,2,1,1,179093.26,False,False,True
1561,632,42,4,119624.60,2,1,1,195978.86,True,False,True
1670,559,24,3,114739.92,1,1,0,85891.02,False,True,True
6087,561,27,9,135637.00,1,1,0,153080.40,False,False,False
6669,517,56,9,142147.32,1,0,0,39488.04,False,False,True


In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense


2026-02-01 14:20:16.458143: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769955616.646533      17 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769955616.702232      17 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769955617.142806      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769955617.142858      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769955617.142861      17 computation_placer.cc:177] computation placer alr

model = Sequential()
model.add(Dense(5, activatonn='relu', input_dim = 11))
model.add(Dense(1, activation='sigmoid'))

In [13]:
model = Sequential() 
model.add(Dense(11, activation='relu', input_dim = 11))
model.add(Dense(11, activation='relu'))
model.add(Dense(11, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2026-02-01 14:20:30.109246: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [14]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 11)             │           132 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 11)             │           132 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 11)             │           132 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            12 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 408 (1.59 KB)

 Trainable params: 408 (1.59 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
 model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
model.fit(X_train_scaled,y_train, epochs=150, validation_split=0.2)

Epoch 1/150
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7732 - loss: 0.5733 - val_accuracy: 0.7987 - val_loss: 0.4663
Epoch 2/150
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7928 - loss: 0.4719 - val_accuracy: 0.7987 - val_loss: 0.4453
Epoch 3/150
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7962 - loss: 0.4492 - val_accuracy: 0.7987 - val_loss: 0.4354
Epoch 4/150
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8025 - loss: 0.4357 - val_accuracy: 0.8050 - val_loss: 0.4296
Epoch 5/150
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7945 - loss: 0.4408 - val_accuracy: 0.8156 - val_loss: 0.4250
Epoch 6/150
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8135 - loss: 0.4236 - val_accuracy: 0.8269 - val_loss: 0.4158
Epoch 7/150
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8240 - loss: 0.4178 - val_accuracy: 0.8388 - val_loss: 0.4008
Epoch 8/150
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8354 - loss: 0.3984 - val_accu

In [17]:
model.layers[0].get_weights()

[array([[-0.2589462 , -0.32183415,  0.04974864, -0.15474795,  0.11320266,
          0.02246149, -0.11425243,  0.0127172 ,  0.06953253, -0.07774895,
          0.03708605],
        [-0.5330106 , -0.31824017,  0.2859466 ,  0.03536585, -0.8759541 ,
         -1.0396686 ,  0.43566188, -0.27106673,  0.16014454, -0.4346185 ,
          0.7798643 ],
        [-0.09172166, -0.15134665,  0.2745778 , -0.05881894, -0.00975476,
         -0.08462567,  0.08096074,  0.00756911, -0.05700609, -0.10631294,
          0.05534592],
        [ 0.85420215,  0.24158184,  0.12099331,  0.77359724,  0.02910848,
         -0.24948017, -0.6739596 , -0.4455397 ,  0.47830382, -0.5620959 ,
          0.5490478 ],
        [-0.28244463,  0.5311642 , -0.5075552 ,  0.9646618 , -0.09154711,
         -0.1176312 ,  0.40717348, -1.0987326 ,  0.23360094,  0.01876907,
          0.17530847],
        [-0.0545758 , -0.18919946,  0.27983004,  0.02393658,  0.03524818,
         -0.08812489,  0.28827608,  0.00600259,  0.3417461 ,  0.0669852

In [18]:
model.layers[1].get_weights()

[array([[-0.36015832,  0.55491227, -0.15439834,  0.14470701, -0.5773489 ,
          0.3334534 , -0.37261572, -0.39026427, -0.03765343, -0.41092885,
         -0.04667181],
        [-0.5245153 ,  0.2770429 ,  0.62587696, -0.3717763 ,  0.6218588 ,
         -0.17102958, -0.04262597, -0.33047557, -0.06331968,  0.17974263,
         -1.270934  ],
        [-0.59869456,  0.16307342,  0.4958579 ,  0.11962681,  0.08334745,
         -0.09561192, -0.7012396 , -0.05165735,  0.13897651,  0.13488035,
          0.544551  ],
        [ 0.11798082, -0.07897531,  0.42800948, -0.5389342 , -0.29659468,
         -1.5710002 ,  0.9571453 ,  0.3616436 , -0.05305021, -0.5399193 ,
         -0.14783579],
        [-0.10975245,  0.59690934, -0.1241823 , -0.00984618, -0.1322323 ,
         -0.3449314 ,  0.27799994,  0.33062965, -0.854305  ,  0.46803424,
          0.33273822],
        [-0.35670465, -0.24274407,  0.90868515,  0.5089648 ,  0.34058237,
         -0.00198124, -0.05479267,  0.36037037, -0.47764227,  0.2546198

In [19]:
y_log = model.predict(X_test_scaled)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  


In [20]:
y_pred = np.where(y_log>=0.5,1,0)

In [21]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.8565